In [2]:
# Access Neo4J credentials
NEO4J_URI=""
NEO4J_USERNAME=""
NEO4J_PASSWORD=""

In [3]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

In [4]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, DOB: INTEGER, name: STRING}
Company {name: STRING}
Developer {POB: STRING, name: STRING, YOB: INTEGER}
Country {name: STRING}
Person {name: STRING, Born: INTEGER}
Movie {name: STRING, Released: INTEGER, id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [5]:
# Load env data
import os
from dotenv import load_dotenv
load_dotenv()

# Groq API Key
groq_api_key = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x73750c1acd40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x73750c1ad610>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(llm=llm, 
                                    graph=graph, 
                                    verbose=True, 
                                    exclude_types=["Genre"], 
                                    allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x73752f54a690>, cypher_generation_chain=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x73750c1acd40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x73750c1ad610>, model_name='gemma2-9b-it', model_kwargs

In [9]:
print(chain.graph_schema)

Node properties are the following:
CEO {POB: STRING, DOB: INTEGER, name: STRING},Company {name: STRING},Developer {POB: STRING, name: STRING, YOB: INTEGER},Country {name: STRING},Person {name: STRING, Born: INTEGER},Movie {name: STRING, Released: INTEGER, id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Relationship properties are the following:

The relationships are the following:
(:Person)-[:ACTED_IN]->(:Movie),(:Person)-[:DIRECTED]->(:Movie)


In [10]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [13]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt=PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt=FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question","schema"]
)

print(prompt)

input_variables=['question'] input_types={} partial_variables={} examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}] example_prompt=PromptTemplate(input_variables=['query', 'question'], 

In [16]:
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,cypher_prompt=prompt,verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x73752f54a690>, cypher_generation_chain=FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_

In [19]:
chain.invoke({"query": "Which actors played in the movie Casino?"})



> Entering new GraphCypherQAChain chain...


KeyError: 'name'